In [1]:
import random
import numpy as np

transp_key = "emirube"
n_columns = len(transp_key)
n_rows = 0

IP_order = [1,5,2,0,3,7,4,6]

random_list = random.sample(range(7), 7)
print(random_list)

key = [str(random.randint(0, 1)) for _ in range(10)]
random_key = ''.join(key)
print(random_key)

def MR(arr, order):
    # Initialize an empty string to hold the resulting text
    result_string = ""
    
    # Iterate through the columns based on the provided order
    for col in order:
        # Extract the entire column and append the values row by row
        for row in arr[:, col]:
            result_string += row  # Concatenate each element
    
    return result_string  

def matrixmaker(text):
    # Initialize the matrix with empty strings
    matrix = np.full((n_rows, n_columns), '', dtype='<U1')

    # Fill the matrix row by row
    for i in range(n_rows):
        # Slice the text list to get the next row
        start_idx = i * n_columns
        end_idx = start_idx + n_columns
        row = text[start_idx:end_idx]  # Get the characters for the current row
        
        # Assign the sliced row to the matrix
        matrix[i, :len(row)] = list(row)  # Only fill as many columns as the row has characters
    
    return matrix

def shift_rows(matrix):
    n_rows, n_columns = matrix.shape
    
    # Create a new matrix to store the result
    shifted_matrix = np.empty_like(matrix)
    
    for i in range(n_rows):
        # Calculate the shift for the i-th row (shift by i positions)
        shift = i % n_columns  # This ensures we don't shift more than the number of columns
        shifted_matrix[i] = np.roll(matrix[i], -shift)  # Shift to the left by 'shift' positions
    
    # Post-processing to remove '' values and shift elements left
    for i in range(n_rows):
        non_empty_values = [val for val in shifted_matrix[i] if val != '']  # Filter non-empty values
        shifted_matrix[i, :len(non_empty_values)] = non_empty_values  # Shift them left
        shifted_matrix[i, len(non_empty_values):] = ''  # Fill the remaining positions with ''

    return shifted_matrix

def permute_8(binary_string):
    # Perform the permutation according to the permutation table
    return ''.join([binary_string[i] for i in IP_order])

[2, 5, 6, 3, 1, 4, 0]
0101001101


In [2]:
plaintext = input("Enter plaintext:")
plaintext = plaintext.replace(" ", "")

ptcode = [char for char in plaintext]

In [3]:
# Calculate the number of rows needed based on the pre-shared number of columns and text
n_rows = (len(ptcode) + n_columns - 1) // n_columns  # Ceiling division

matrix = matrixmaker(ptcode)
print(matrix)
firstround = MR(matrix, random_list)
print("First round:", firstround)
print()

smatrix = matrixmaker(firstround)
print(smatrix)
secondround = MR(smatrix, random_list)
print("Second round:", secondround)
print()

tmatrix = matrixmaker(secondround)
print(tmatrix)
thirdround = MR(tmatrix, random_list)
print("Third round:", thirdround)

[['a' 'r' 'e' 'y' 'o' 'u' 'c']
 ['o' 'm' 'i' 'n' 'g' 't' 'o']
 ['t' 'h' 'e' 't' 'o' 'g' 'a']
 ['p' 'a' 'r' 't' 'y' '' '']]
First round: eierutgcoaynttrmhaogoyaotp

[['e' 'i' 'e' 'r' 'u' 't' 'g']
 ['c' 'o' 'a' 'y' 'n' 't' 't']
 ['r' 'm' 'h' 'a' 'o' 'g' 'o']
 ['y' 'a' 'o' 't' 'p' '' '']]
Second round: eahottggtoryatiomaunopecry

[['e' 'a' 'h' 'o' 't' 't' 'g']
 ['g' 't' 'o' 'r' 'y' 'a' 't']
 ['i' 'o' 'm' 'a' 'u' 'n' 'o']
 ['p' 'e' 'c' 'r' 'y' '' '']]
Third round: homctangtooraratoetyuyegip


In [4]:
trans_output = matrixmaker(thirdround)
print("Transposition output:")
print(trans_output)
print()

shifted_output = shift_rows(trans_output)
print("Shifted output:")
print(shifted_output)

Transposition output:
[['h' 'o' 'm' 'c' 't' 'a' 'n']
 ['g' 't' 'o' 'o' 'r' 'a' 'r']
 ['a' 't' 'o' 'e' 't' 'y' 'u']
 ['y' 'e' 'g' 'i' 'p' '' '']]

Shifted output:
[['h' 'o' 'm' 'c' 't' 'a' 'n']
 ['t' 'o' 'o' 'r' 'a' 'r' 'g']
 ['o' 'e' 't' 'y' 'u' 'a' 't']
 ['i' 'p' 'y' 'e' 'g' '' '']]


In [5]:
# Flatten the shifted matrix into a long list
flattened_list = shifted_output.flatten().tolist()

# Remove all '' values
filtered_list = [val for val in flattened_list if val != '']

# Print the flattened list
print(filtered_list)

['h', 'o', 'm', 'c', 't', 'a', 'n', 't', 'o', 'o', 'r', 'a', 'r', 'g', 'o', 'e', 't', 'y', 'u', 'a', 't', 'i', 'p', 'y', 'e', 'g']


In [6]:
binary_list = [format(ord(char), '08b') for char in filtered_list]
print(binary_list)

['01101000', '01101111', '01101101', '01100011', '01110100', '01100001', '01101110', '01110100', '01101111', '01101111', '01110010', '01100001', '01110010', '01100111', '01101111', '01100101', '01110100', '01111001', '01110101', '01100001', '01110100', '01101001', '01110000', '01111001', '01100101', '01100111']


In [7]:
p8 = permute_8(binary_list[0])
print(p8)

10100010
